##### Here we will do all the data exploration which are more bitcoin specific and requires more knowledge about bitcoin ->


In [2]:
import os
import json
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Work.Helper import list_all_tx  # Import the function from Script.py
from Work.Script import *
# import Opcodes
# from Work.Helper import little_endian_to_int,list_all_tx,int_to_little_endian

all_files= list_all_tx() 

In [3]:
import sys
sys.path

['/home/divyanshgupta/code-challenge-2024-15IITian/Work/Data_Enginerring',
 '/usr/lib64/python311.zip',
 '/usr/lib64/python3.11',
 '/usr/lib64/python3.11/lib-dynload',
 '',
 '/home/divyanshgupta/code-challenge-2024-15IITian/.venv/lib64/python3.11/site-packages',
 '/home/divyanshgupta/code-challenge-2024-15IITian/.venv/lib/python3.11/site-packages',
 '/home/divyanshgupta/code-challenge-2024-15IITian']

## Analysis of txns including `P2SH`->


In [4]:
witness_empty=0
witness_empty_file=[]
redeem_witness_empty=[]
redeem_empty_witness_present=[]
redeem_present_witness_present=0
psh=0
redeem_present_witness_empty= []

for file in all_files:
   
        for input in file['vin']:
         
            if input['prevout']['scriptpubkey_type']== 'p2sh':
               psh+=1
               if input.get("witness")==None:
                    witness_empty+=1
                    witness_empty_file.append(file)
                    if input.get("inner_redeemscript_asm")==None:
                         redeem_witness_empty.append(file)
                         
               if input.get("witness")==None:
                    witness_empty+=1
                    witness_empty_file.append(file)
               if input.get("witness") != None and input.get("inner_redeemscript_asm")==None:
                    redeem_empty_witness_present.append(file)

               if input.get("witness") != None: 
                  if input.get("inner_redeemscript_asm")!=None:
                     redeem_present_witness_present+=1   
               if input.get("witness") == None and input.get("inner_redeemscript_asm")==None:
                    redeem_present_witness_empty.append(file)
                      


# print(witness_empty)                    
                   
# print(witness_empty_file[0])
print(redeem_witness_empty)

print(redeem_empty_witness_present)
print(redeem_present_witness_present)
print(psh)
print(len(redeem_present_witness_empty))


              


[]
[]
3634
3672
0


##### Observation ->

- If in all tx script type -> there is no mention of `p2sh-p2wkh` and `p2sh-p2wsh` script type.
- Note : `P2sh` -> Total = 3672 , `p2sh` = 38 , nested_p2sh = rest


### Idea to segreate them -> 
  - `p2sh` -> no witness  + yes(inner_redeem_script)
  - `p2sh-p2wpkh` -> witness + inner_redeem_script - inner_witness_script
  - `p2sh-p2wsh` -> witness + inner_redeem_script + inner_witness_script


In [6]:
folder_path = "./mempool"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for section in ["vin","vout"]:
      for input in data[section]:
        if section == "vin":
           script_type = input["prevout"]["scriptpubkey_type"]
        else:
           script_type = input["scriptpubkey_type"]
           
              
           
        
        if script_type == "unknown":
            output["scriptpubkey_type"] = "bare-multisig"
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

SyntaxError: invalid syntax (2815966488.py, line 10)

**Check** witness field should not exist and script_sig not empty for legacy address and vice versa for segwit script


['OP_PUSHNUM_1',
 'OP_PUSHBYTES_32',
 '727da28731cccc6f227d0200b43c384ae06e717f6f14f62f31145a8cbb145048']

- Check whether `parse(sript_pubkey)` == `scriptpubkey_asm` or not ?
- `parse(sript_sig)` == `scriptsig_asm`?
- Find all opcodes used in the script for definign the function
-


In [ ]:
script_pubkey_unmatch=[]
script_sig_unmatch=[]
all_opcodes_used = set() 
for file_index, file in enumerate(all_files):
    for section in ["vout","vin"]:
        for input_index, input in enumerate(file[section]):
            scriptpubkey_asm=""
            parsed_script_pubkey=""
            parsed_script_sig=""
            scriptsig_asm=""
            if section == "vin":
                parsed_script_pubkey=  Script.parse(input["prevout"]["scriptpubkey"])
                parsed_script_sig= Script.parse(input["scriptsig"])
                scriptpubkey_asm= input["prevout"]["scriptpubkey_asm"]
                scriptsig_asm= input["scriptsig_asm"]
            else:
                parsed_script_pubkey= Script.parse(input["scriptpubkey"])
                scriptpubkey_asm= input["scriptpubkey_asm"]

            if str(parsed_script_pubkey)!= scriptpubkey_asm:
                script_pubkey_unmatch.append([file_index,section])    
            if str(parsed_script_sig) !=  scriptsig_asm:
                script_sig_unmatch.append([file_index,section,input_index]) 
            # finding all the opcodes used in the script ->     
            for script_type in [scriptpubkey_asm,scriptsig_asm]:
                for word in script_type.split():
                    if word[:2] == "OP":
                        all_opcodes_used.add(word)




print(len(script_pubkey_unmatch))
print("script_pubkey_unmatch : {}".format(script_pubkey_unmatch))
print("script_sig_unmatch:{}".format(script_sig_unmatch))  
print("all opcodes used : {}".format(all_opcodes_used))              

                


0
script_pubkey_unmatch : []
script_sig_unmatch:[]
all opcodes used : {'OP_PUSHBYTES_32', 'OP_PUSHBYTES_34', 'OP_PUSHBYTES_65', 'OP_PUSHBYTES_31', 'OP_DUP', 'OP_PUSHBYTES_42', 'OP_PUSHBYTES_74', 'OP_PUSHBYTES_72', 'OP_PUSHBYTES_12', 'OP_PUSHBYTES_9', 'OP_PUSHBYTES_13', 'OP_PUSHBYTES_11', 'OP_PUSHDATA1', 'OP_CHECKSIG', 'OP_CHECKMULTISIG', 'OP_PUSHNUM_3', 'OP_PUSHBYTES_69', 'OP_PUSHBYTES_46', 'OP_PUSHBYTES_73', 'OP_PUSHBYTES_64', 'OP_PUSHBYTES_4', 'OP_PUSHBYTES_17', 'OP_HASH160', 'OP_PUSHBYTES_1', 'OP_PUSHDATA2', 'OP_EQUALVERIFY', 'OP_PUSHBYTES_22', 'OP_PUSHBYTES_67', 'OP_PUSHNUM_1', 'OP_PUSHBYTES_33', 'OP_PUSHBYTES_20', 'OP_EQUAL', 'OP_PUSHBYTES_71', 'OP_PUSHBYTES_10', 'OP_PUSHBYTES_54', 'OP_0', 'OP_PUSHBYTES_63', 'OP_PUSHBYTES_70', 'OP_PUSHBYTES_68', 'OP_RETURN', 'OP_PUSHBYTES_5'}


41

#### Observation ->

- `Scriptpubkey` matches with `Scriptpubkey_asm` + script_case for all txn
- Total 41 opcodes are used -> `all_opcodes_used`


In [ ]:
a = "OP_PUSHNUM_1"
a[:2]

'OP'